In [1]:
import numpy as np
import pandas as pd

from ema_workbench import load_results, save_results, TimeSeriesOutcome

In [ ]:
no_flood = False # ADJUST HERE 

nfl = ""
if no_flood:
    nfl = "no_flood/"

In [2]:
path = f'../results/{nfl}/'
file_name = '2000_scen__40_reps__0703' # ADJUST HERE
file_ext  = '.tar.gz'

In [3]:
experiments, outcomes = load_results(path+file_name+file_ext)

In [4]:
# Set modes
burn_in = True
resample = False
add_rates = False

In [5]:
# Can use any number of the original OOIs. 
# This list essentially shrinks the output variables considered.
oois = [
    'Household Population',
    'GDP',
    'Gini Coefficient',
    'Median Wage',
] 

In [ ]:
# If you want to use all measured outcomes, uncomment the next line :
# oois = [k for k in outcomes]

### Burn-in

Remove first 5 years as burn-in time (migration cannot occur)

In [6]:
if burn_in:
    for ooi in oois:
        outcomes[ooi] = [run[20:] for run in outcomes[ooi]]

In [7]:
len(outcomes[oois[0]][0])

100

### Resample
Collect a sample each year, instead of each quarter

In [8]:
if resample:
    for ooi in oois:
        outcomes[ooi] = [run[::4] for run in outcomes[ooi]]

In [9]:
len(outcomes[oois[0]][0])

100

### Add rates

Add growth rates as outcomes of their own

In [10]:
if add_rates:
    rates = {ooi:[] for ooi in oois}
    for ooi in oois:
        for run in outcomes[ooi]:
            rate_run = [0]
            for t in range(1,len(run)):
                rate_run.append(run[t] - run[t-1])
            
            # Save back as a Numpy array
            # rate_out = np.array(rate_out)
            rates[ooi].append(rate_run)
        
        rates[ooi] = np.array(rates[ooi])

        ema_outcome = TimeSeriesOutcome(f"Change in {ooi}")
        outcomes[f"Change in {ooi}"] = rates[ooi]

In [12]:
if add_rates:
    print(len(outcomes[f'Change in {oois[0]}'][0]))

In [13]:
results = experiments, outcomes

fn = path + file_name
if burn_in:
    fn += '__burned_in'
if resample:
    fn += '__resampled'
if add_rates:
    fn += '__with_rates'
    
save_results(results, fn + file_ext)

c:\Users\Gabriel\Documents\Academic\EPA\2. Thesis\Code\CRAB_EM\.venv\Lib\site-packages\ema_workbench\em_framework\outcomes.py:540: UserWarning: still to be tested!!
  warnings.warn("still to be tested!!")
